In [1]:
import pandas as pd
import sys
sys.path.append('/')
from utils_processing import transform_verbose_to_calendar
from utils_plot import plot_countries_been_over_years, create_map

In [2]:
df_verbose = pd.read_csv('../data/trips_verbose.csv')
df = transform_verbose_to_calendar(df_verbose, save=False)
df = df[df['country'] != 'All']
plot_countries_been_over_years(df)

In [3]:
VAR = 'total_days'

create_map(
    df, 
    var=VAR, 
    code_convention='code3', 
    bins=[0, 7, 30, 365, df[VAR].max()],
    color='royalblue',
    projection_type="orthographic", # orthographic, equirectangular, mercator, natural earth, kavrayskiy7, miller, robinson, eckert4, azimuthal equal area, azimuthal equidistant, gnomonic, stereographic, mollweide
    tooltip_mode='calendar',
    #save_path='../attachments/map_travel.html'
    save_path=None
    )